# <p style="text-align: center;">Aula Prática 05</p>

## Objetivos desta aula:

    - Segmentação


## Professor: Jefersson dos Santos - jefersson@dcc.ufmg.br
### Monitor: Caio Cesar - caiosilva@ufmg.br

Importações necessárias

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi
from ipywidgets import widgets
from skimage import img_as_float, segmentation, color, io
from skimage.data import coins, binary_blobs, astronaut, page, camera, coffee
from skimage.segmentation import clear_border, random_walker, felzenszwalb, slic, quickshift, watershed, mark_boundaries
from skimage.morphology import label, closing, square, watershed, disk
from skimage.measure import regionprops
from skimage.color import lab2rgb, rgb2gray
from skimage.filters import rank, sobel, threshold_otsu, try_all_threshold
from skimage.util import img_as_ubyte, img_as_float
from skimage.future import graph
from skimage.exposure import rescale_intensity

%matplotlib inline

Comparação dos métodos de Threshold

In [ ]:
img = page()

# Here, we specify a radius for local thresholding algorithms.
# If it is not specified, only global algorithms are called.
fig, ax = try_all_threshold(img, figsize=(10, 8), verbose=False)
plt.show()

Função auxiliar para mostrar as images:

In [ ]:
def show(img, cmap=None):
    cmap = cmap or plt.cm.gray
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.imshow(img, cmap=cmap)
    ax.set_axis_off()
    plt.show()

Usando imagem das moedas

In [ ]:
imgC = coins()
show(imgC)

Calculando o Threshold Otsu e aplicando na imagem

In [ ]:
thr = threshold_otsu(imgC)
print "Threshold otsu: "+str(thr)
show(imgC > thr)

Encontrando um valor mais adequado

In [ ]:
@widgets.interact(t=(50, 240))
def threshold(t):
    show(imgC > t)

Utilizando transformações morfológicas para melhorar a segmentação.

In [ ]:
img_bin = clear_border(closing(imgC > 120, square(5)))
show(img_bin)

Utilizando o Watershed

In [ ]:
image = img_as_ubyte(camera())

# denoise image
denoised = rank.median(image, disk(2))

# find continuous region (low gradient - where less than 10 for this image) --> markers
# disk(5) is used here to get a more smooth image
markers = rank.gradient(denoised, disk(5)) < 10
markers = ndi.label(markers)[0]

# local gradient (disk(2) is used to keep edges thin)
gradient = rank.gradient(denoised, disk(2))

# process the watershed
labels = watershed(gradient, markers)

print ("Original")
show(image)
print ("Local Gradient")
show(gradient, cmap=plt.cm.nipy_spectral)
print ("Markers")
show(markers, cmap=plt.cm.nipy_spectral)
print ("Segmented")
show(labels, cmap=plt.cm.nipy_spectral)

Utilizando SLIC

In [ ]:
img = coffee()

labels1 = segmentation.slic(img, compactness=30, n_segments=400)
out1 = color.label2rgb(labels1, img, kind='avg')

g = graph.rag_mean_color(img, labels1)
labels2 = graph.cut_threshold(labels1, g, 29)
out2 = color.label2rgb(labels2, img, kind='avg')

g3 = graph.rag_mean_color(img, labels1, mode='similarity')
labels3 = graph.cut_normalized(labels1, g)
out3 = color.label2rgb(labels3, img, kind='avg')

show(out1)
show(out2)
show(out3)


Usando Random Wlaker

In [ ]:
# Generate noisy synthetic data
data = img_as_float(binary_blobs(length=128, seed=18))
sigma = 0.35
data += np.random.normal(loc=0, scale=sigma, size=data.shape)
data = rescale_intensity(data, in_range=(-sigma, 1 + sigma),
                         out_range=(-1, 1))

# The range of the binary image spans over (-1, 1).
# We choose the hottest and the coldest pixels as markers.
markers = np.zeros(data.shape, dtype=np.uint)
markers[data < -0.95] = 1
markers[data > 0.95] = 2

# Run random walker algorithm
labels = random_walker(data, markers, beta=10, mode='bf')

show(data)
show(markers)
show(labels)

Comparação entre o uso de segmentação e superpixel

In [ ]:
def shoeImgMArkers():
    fig, ax = plt.subplots(1, 1, figsize=(10, 10), sharex=True, sharey=True)
    ax[0, 0].imshow(mark_boundaries(img, segments_fz))
    ax[0, 0].set_title("Felzenszwalbs's method")

In [ ]:
img = img_as_float(astronaut()[::2, ::2])

segments_fz = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
segments_slic = slic(img, n_segments=250, compactness=10, sigma=1)
segments_quick = quickshift(img, kernel_size=3, max_dist=6, ratio=0.5)
gradient = sobel(rgb2gray(img))
segments_watershed = watershed(gradient, markers=250, compactness=0.001)

print("Felzenszwalb number of segments: {}".format(len(np.unique(segments_fz))))
print('SLIC number of segments: {}'.format(len(np.unique(segments_slic))))
print('Quickshift number of segments: {}'.format(len(np.unique(segments_quick))))

print "Felzenszwalbs's method"
show(mark_boundaries(img, segments_fz))
print "SLIC"
show(mark_boundaries(img, segments_slic))
print "Quickshift"
show(mark_boundaries(img, segments_quick))     
print "Compact watershed"
show(mark_boundaries(img, segments_watershed))

Classificação simples

In [ ]:
labels = label(img_bin)
show(labels, cmap=plt.cm.rainbow)
regions = regionprops(labels)
boxes = np.array([label['BoundingBox']
                  for label in regions
                  if label['Area'] > 100])
print("Existem "+str(len(boxes))+" moedas.")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.imshow(imgC, cmap=plt.cm.gray)
ax.set_axis_off()

# Get the coordinates of the boxes.
xs = boxes[:, [1, 3]].mean(axis=1)
ys = boxes[:, [0, 2]].mean(axis=1)

# We reorder the boxes by increasing
# column first, and row second.
for row in range(4):
    # We select the coins in each of the four rows.
    if row < 3:
        ind = ((ys[6 * row] <= ys) &
               (ys < ys[6 * row + 6]))
    else:
        ind = (ys[6 * row] <= ys)
    # We reorder by increasing x coordinate.
    ind = np.nonzero(ind)[0]
    reordered = ind[np.argsort(xs[ind])]
    xs_row = xs[reordered]
    ys_row = ys[reordered]
    # We display the coin number.
    for col in range(6):
        n = 6 * row + col
        ax.text(xs_row[col] - 5, ys_row[col] + 5,
                str(n),
                fontsize=20)

# Exercício

    Crie um novo notebook jupyter cujo nome deve seguir a seguinte syntaxe:
    
        lab5-SUAMATRICULA.ypnb

    Escolha uma imagem em escala de cinza e uma colorida.
    Escreva o código para carregar e mostrar a imagem.
    Escreva o código e mostre uma segmentação simples usando threshold.
    Escreva o código e mostre uma segmentação watershed.
    Escreva o código e mostre uma segmentação usando SLIC.
    Comente sobre os resultados obtidos com cada um dos métodos e como eles se comparam.
        